### Change Log
---
Restructuring code, hadelling parameters

In [1]:
import sys
sys.path.append('/data/extensibleFramework/')

In [2]:
from extensibleFramework.src.models import convolution_neural_network
from extensibleFramework.src.models import recurrent_nn_with_attention
from extensibleFramework.src.models import feed_forward_network
from extensibleFramework.src.models import extra_layers
from extensibleFramework.src.utils import saving_and_loading
from extensibleFramework.src.models import config

In [3]:
import pandas as pd
import chakin
import matplotlib.pyplot as plt
from torchtext import data
import nltk
import json
from torchtext import vocab
from tqdm import tqdm
import torch
import random
import re
import pandas as pd
import os
import numpy as np
import sys
import random
import tarfile
import urllib
from torchtext import data
import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_frame = pd.read_csv("labeledTrainData.tsv",sep="\t")
data_frame = data_frame[['review', 'sentiment']]
data_block  = data_frame.values

In [5]:
SEED = 1
split = 0.80

In [6]:
random.shuffle(data_block)

# train_file = open('train.json', 'w')
# test_file = open('test.json', 'w')
# for i in  range(0,int(len(data_block)*split)):
#     train_file.write(str(json.dumps({'review' : data_block[i][0], 'label' : data_block[i][1]}))+"\n")
# for i in  range(int(len(data_block)*split),len(data_block)):
#     test_file.write(str(json.dumps({'review' : data_block[i][0], 'label' : data_block[i][1]}))+"\n")

In [7]:
def tokenize(sentiments):
    
    return nltk.tokenize.word_tokenize(sentiments)
# def pad_to_equal(x):
#     if len(x) < 61:
#         return x + ['<pad>' for i in range(0, 61 - len(x))]
#     else:
#         return x[:61]
def to_categorical(x):
    x = int(x)
    if x == 1:
        return [0,1]
    if x == 0:
        return [1,0]

In [8]:
SENTIMENT = data.Field(sequential=True , tokenize=tokenize, use_vocab = True, lower=True,batch_first=True)
LABEL = data.Field(is_target=True,use_vocab = False, sequential=False, preprocessing = to_categorical)
fields = {'sentiment': ('sentiment', SENTIMENT), 'label': ('label', LABEL)}
train_data , test_data = data.TabularDataset.splits(
                            path = '',
                            train = 'train.json',
                            test = 'test.json',
                            format = 'json',
                            fields = fields)


In [9]:
# chakin.search(lang='English')

In [10]:
# chakin.download(number=12, save_dir = "./")

In [11]:
# !unzip glove.6B.zip

In [12]:
vec = vocab.Vectors(name = "glove.6B.100d.txt",cache = "./")

In [13]:
SENTIMENT.build_vocab(train_data, test_data, max_size=400000, vectors=vec)

In [14]:
sentiment_vocab = SENTIMENT.vocab

In [15]:
# defining config object 
cnn_rnn_vocab_size = len(SENTIMENT.vocab)
cnn_rnn_embed_dim = 100
cnn_rnn_class_num = 50
out_channel_num = 8
cnn_kernel_sizes = [3,4,5]
rnn_n_layers = 1
rnn_hidden_size = 128
use_pretrained_weights = True
cnn_rnn_weights = sentiment_vocab.vectors
cnn_rnn_weight_is_trainable =  False
dropout = 0.2
batch_size = 32
merge_mode = "CONCAT"
ffn_activation = "Relu"
ffn_final_output_classes = 2
ffn_num_layer = 2 
ffn_perceptron_per_layer = [50, 25]
ffn_layer_wise_dropout = [0.2,0.2]
device = device

config = config.Config(cnn_rnn_vocab_size, cnn_rnn_embed_dim, cnn_rnn_class_num, out_channel_num, \
        cnn_kernel_sizes, rnn_n_layers, rnn_hidden_size, use_pretrained_weights,cnn_rnn_weights, cnn_rnn_weight_is_trainable,\
            dropout, batch_size, merge_mode, ffn_activation, ffn_final_output_classes, ffn_num_layer, ffn_perceptron_per_layer,\
                       ffn_layer_wise_dropout, device)


In [16]:
train_iter, test_iter = data.Iterator.splits(
        (train_data, test_data), sort_key=lambda x: len(x.sentiment),
        batch_sizes=(batch_size,batch_size), device=device)

In [17]:
for batch in test_iter:
    feature, target = batch.sentiment, batch.label
    print(feature.data.shape, target.data.shape)
    break

torch.Size([32, 46]) torch.Size([32, 2])


In [18]:
sentiment_vocab.vectors.shape

torch.Size([118522, 100])

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class ensemble_model(nn.Module):
    def __init__(self, config_object):
        super(ensemble_model, self).__init__() 
        self.cnn = convolution_neural_network.cnn_text(config_object)
        self.rnn_attention = recurrent_nn_with_attention.RNNAttentionModel(config_object)
        self.merge_layer = extra_layers.MergeAndFlattern(config_object)
        self.ffn = feed_forward_network.ffn(config_object)
    def forward(self, x):
        cnn_output = self.cnn(x)
        rnnAttention_output = self.rnn_attention(x)
        merge_layer_output = self.merge_layer(cnn_output,rnnAttention_output )
        final_output = self.ffn(merge_layer_output)
        
#         print("cnn_output : ",cnn_output.shape)
#         print("rnn_input : ",rnnAttention_output.shape)
#         print("merge_layer_output : ", merge_layer_output.shape)
#         print("final_output : ", final_output.shape)
        return final_output

In [20]:
EM  = ensemble_model(config)
EM  = EM.to(device)

In [21]:
cnn_input = torch.Tensor(np.random.randint(5, size=[8, 10])).long().to(device)
rnn_input = torch.Tensor(np.random.random([8, 10])).long().to(device) #batch_size, input_size

In [22]:
# EM(rnn_input)

In [23]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded_preds = torch.argmax(preds, dim=1)
    correct = (rounded_preds == torch.argmax(y, dim=1)).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [24]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        feature, target = batch.sentiment, batch.label
        optimizer.zero_grad()
        predictions = model(feature.to(device))            
        loss = criterion(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        loss.backward()
        optimizer.step()
        acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return model, epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
def test_accuracy_calculator(model, test_iterator):
    epoch_acc = 0
    for batch in test_iterator:
        if batch.sentiment.shape[0] ==  32:
            feature, target = batch.sentiment, batch.label
            predictions = model(feature.to(device))            
            acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
            epoch_acc += acc.item()
    return  epoch_acc / len(test_iterator)

In [ ]:
optimizer = torch.optim.SGD(EM.parameters(), lr=0.1,momentum=0.9)
criterion = nn.MSELoss()
criterion = criterion.to(device)

In [ ]:
epochs  = 100
log_interval = 1
loss = []
accuracy = []
test_accuracy = []
for i in tqdm(range(epochs)):
    if (i != 0 and i%10 == 0 ):
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr']/2
        print (" === New Learning rate : ", param_group['lr'], " === ")
    
    model, epoch_loss, epoch_acc = train(EM, train_iter, optimizer, criterion)
    
    test_acc = test_accuracy_calculator(model, test_iter)
    accuracy.append(epoch_acc)
    loss.append(epoch_loss)
    test_accuracy.append(test_acc)
    print(epoch_acc,test_acc,epoch_loss)

  1%|          | 1/100 [00:45<1:15:52, 45.98s/it]

0.50305 0.4964171974522293 0.25141215829849245


  2%|▏         | 2/100 [01:31<1:15:05, 45.97s/it]

0.49855 0.4964171974522293 0.25117810282707215


  3%|▎         | 3/100 [02:17<1:14:12, 45.90s/it]

...

0.91405 0.8531050955414012 0.06876057477518917


 32%|███▏      | 32/100 [24:18<51:40, 45.59s/it]

0.9132 0.8554936305732485 0.06930147396847605


 33%|███▎      | 33/100 [25:04<50:49, 45.51s/it]

0.91455 0.8552945859872612 0.0682074833586812


In [ ]:
SAL = saving_and_loading.objectManager()

In [ ]:
SAL.saver(EM, "./EM.ckpt")

In [ ]:
EM = SAL.loader("./EM.ckpt")